# Capstone East Valley Reviews Sentiment Analysis (ngram_range = 1,3)

### Imports

In [1]:
import os
import re
import io
import requests
import pandas as pd
import emojis
import string
import spacy
import nltk

In [2]:
url = "https://raw.githubusercontent.com/laurakaneta/East_Valley_Reviews/main/East_Valley_Reviews_Data_Deidentified.csv"
downloaded = requests.get(url).content
EastValleyFile = pd.read_csv(io.StringIO(downloaded.decode('ISO-8859-1')))
EastValleyFile

,ï»¿query,name,google_id,place_id,location_link,reviews_link,reviews_per_score,rating,review_id,autor_link,...,owner_answer,owner_answer_timestamp,owner_answer_timestamp_datetime_utc,review_link,review_rating,review_timestamp,review_datetime_utc,review_likes,reviews_id,City
0,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSURhbzVtaTZ3RRAB,https://www.google.com/maps/contrib/1137410538...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1630274471,08/29/2021 22:01:11,3,5586047838209581501,Gilbert
1,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChZDSUhNMG9nS0VJQ0FnSUNxeWZ1Wk13EAE,https://www.google.com/maps/contrib/1120312345...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,3,1623383810,06/11/2021 03:56:50,0,5586047838209581501,Gilbert
2,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSURxcnFuZWh3RRAB,https://www.google.com/maps/contrib/1080138635...,...,Thank you for coming for dine in with us tonig...,1.625802e+09,07/09/2021 03:38:27,https://www.google.com/maps/reviews/data=!4m5!...,3,1625801519,07/09/2021 03:31:59,0,5586047838209581501,Gilbert
3,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSUNhMnBpMDJnRRAB,https://www.google.com/maps/contrib/1140612880...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1627531836,07/29/2021 04:10:36,1,5586047838209581501,Gilbert
4,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChZDSUhNMG9nS0VJQ0FnSUNhMW9HcUtnEAE,https://www.google.com/maps/contrib/1067087380...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1627687884,07/30/2021 23:31:24,1,5586047838209581501,Gilbert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,Copper Still Moonshine Grill - Gilbert,Copper Still Moonshine Grill - Gilbert,0x872babbf7c5e9241:0xfe116985d756993b,ChIJQZJefL-rK4cRO5lW14VpEf4,https://www.google.com/maps/place/Copper+Still...,https://search.google.com/local/reviews?placei...,"{""1"": 17, ""2"": 16, ""3"": 57, ""4"": 156, ""5"": 420}",4.4,ChZDSUhNMG9nS0VJQ0FnSUR3aEx2WlhnEAE,https://www.google.com/maps/contrib/1159259620...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,4,1539763834,10/17/2018 08:10:34,1,-139214089907431109,Gilbert
10458,Copper Still Moonshine Grill - Gilbert,Copper Still Moonshine Grill - Gilbert,0x872babbf7c5e9241:0xfe116985d756993b,ChIJQZJefL-rK4cRO5lW14VpEf4,https://www.google.com/maps/place/Copper+Still...,https://search.google.com/local/reviews?placei...,"{""1"": 17, ""2"": 16, ""3"": 57, ""4"": 156, ""5"": 420}",4.4,ChdDSUhNMG9nS0VJQ0FnSUN3OHNhNnh3RRAB,https://www.google.com/maps/contrib/1000622061...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1459199210,03/28/2016 21:06:50,1,-139214089907431109,Gilbert
10459,Copper Still Moonshine 

In [3]:
EastValleyFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ï»¿query                             10462 non-null  object 
 1   name                                 10462 non-null  object 
 2   google_id                            10462 non-null  object 
 3   place_id                             10462 non-null  object 
 4   location_link                        10462 non-null  object 
 5   reviews_link                         10462 non-null  object 
 6   reviews_per_score                    10462 non-null  object 
 7   rating                               10462 non-null  float64
 8   review_id                            10462 non-null  object 
 9   autor_link                           10462 non-null  object 
 10  review_text                          10401 non-null  object 
 11  review_img_url              

Since only the reviews and ratings will be used to make the sentiment analysis, all other columns can be dropped and all na values will be dropped as well. 

In [4]:
East_Valley_slimmed_down = EastValleyFile[['review_text', 'review_rating']]
East_Valley_slimmed_down = East_Valley_slimmed_down.dropna()

In [5]:
East_Valley_slimmed_down.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10401 entries, 0 to 10461
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_text    10401 non-null  object
 1   review_rating  10401 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 243.8+ KB


All positive reviews (4 or 5) will be denoted with a 1, all neutral reviews (3) will be denoted with a 0, and all negative reviews (1 or 2) will be denoted with a -1 under the "sentiment" column

In [6]:
sentiment = []
for rating in East_Valley_slimmed_down.review_rating:
    if rating > 3:
        sentiment.append(1)
    elif rating < 3:
        sentiment.append(-1)
    else:
        sentiment.append(0) 
East_Valley_slimmed_down['sentiment'] = sentiment

To Pre-Process the data, all the text will be lowercase and all punctuation will be removed. (CountVectorizer actually performs these steps and tokenizes the data, but I wanted to manually inspect the reviews before using them to train the models). 

In [7]:
East_Valley_slimmed_down['review_text'] = East_Valley_slimmed_down.iloc[:,0].str.lower()

In [8]:
Reviews = East_Valley_slimmed_down['review_text'].copy()

In [9]:
Reviews_updated= Reviews.replace('['+string.punctuation+']', '', regex=True)

In [10]:
East_Valley_slimmed_down= East_Valley_slimmed_down.drop('review_text', axis = 1)
East_Valley_slimmed_down['reviews'] = Reviews_updated

I decided to remove all the reviews with a score of 3 (neutral) for the sentiment analysis. 

In [11]:
East_Valley_Negative_Positive = East_Valley_slimmed_down[East_Valley_slimmed_down.sentiment != 0]
East_Valley_Negative_Positive

,review_rating,sentiment,reviews
0,5,1,what a great find from the outside it may make...
3,5,1,the restaurant has pleasant dining seating arr...
4,5,1,been craving a thai food place since we left o...
5,5,1,amazing food with authentic flavors drunken no...
6,5,1,finally i found my favorite kuey teow nham tok...
...,...,...,...
10457,4,1,cozy atmosphere good lunch and dinner selectio...
10458,5,1,i love the neighborhood vibe of this place its...
10459,5,1,our waitress must have overdosed on vitamin d ...
10460,1,-1,poor service improperly prepared food and over...


In [12]:
East_Valley_Negative_Positive.sentiment.value_counts()

 1    8990
-1     874
Name: sentiment, dtype: int64

In [13]:
Negpercent = round(((East_Valley_Negative_Positive.sentiment.value_counts()[-1]/
                     sum(East_Valley_Negative_Positive.sentiment.value_counts()[:]))
              *100),2)

pospercent = round(((East_Valley_Negative_Positive.sentiment.value_counts()[1]/
                     sum(East_Valley_Negative_Positive.sentiment.value_counts()[:]))
              *100),2)

print(Negpercent, 'percent of the reviews have a negative sentiment, while', '\n', pospercent, 
      'percent of the reviews have a positive sentiment.')

8.86 percent of the reviews have a negative sentiment, while 
 91.14 percent of the reviews have a positive sentiment.


The data is imbalanced. Only 9.72 percent of the reviews are classified as negative sentiment. However, I do not have the ability to scrape more Google Reviews to gain more negative sentiment ones so I am going to traim multiple models and use a confusion matrix anbd a full classification report to look more in-depth at how the models performs. 

Split data to train models

In [14]:
X = East_Valley_Negative_Positive['reviews']
y = East_Valley_Negative_Positive['sentiment']

X[216]

'amazing  breakfast i got the big breakfast and it was great the biscuit was buttery and flaky everything tasted delicious the waffle was a bit soggy as though it had been cooked yesterday and reheated it was too much food so i brought the waffle home and the next morning i toasteded it in my toaster oven and it was great my husband got the chicken and waffles the chicken was amazing its a different sort of spice with a mustard sauce it tasted more like something you would get a highend restaurant'

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cv_doc = CountVectorizer(stop_words = "english", ngram_range=(1, 3))
Xtrain_vec = cv_doc.fit_transform(X_train)
Xtest_vec = cv_doc.transform(X_test)

In [18]:
len(cv_doc.vocabulary_)

245527

In [19]:
cv_doc.vocabulary_

{'corrected': 46227,
 'order': 151562,
 'times': 221496,
 'different': 57508,
 'people': 159055,
 'got': 96051,
 'wrong': 243240,
 'told': 222842,
 'beef': 17976,
 'bowl': 23648,
 'thing': 218396,
 'italian': 114965,
 'just': 117746,
 'pile': 162397,
 'meat': 137620,
 'thats': 217661,
 'giant': 91222,
 'meatball': 137853,
 'super': 210448,
 'disappointing': 59231,
 'corrected order': 46232,
 'order times': 152430,
 'times different': 221591,
 'different people': 57648,
 'people got': 159268,
 'got wrong': 96946,
 'wrong told': 243374,
 'told beef': 222871,
 'beef bowl': 17993,
 'bowl thing': 23810,
 'thing italian': 218514,
 'italian bowl': 114980,
 'bowl beef': 23664,
 'bowl just': 23747,
 'just pile': 118585,
 'pile meat': 162402,
 'meat thats': 137832,
 'thats italian': 217739,
 'bowl pile': 23768,
 'pile beef': 162398,
 'beef giant': 18060,
 'giant meatball': 91239,
 'meatball super': 137870,
 'super disappointing': 210549,
 'corrected order times': 46233,
 'order times different':

## Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
logreg= LogisticRegression()
logreg.fit(Xtrain_vec,y_train)
preds = logreg.predict(Xtest_vec)

In [21]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(preds,y_test)

array([[  92,    6],
       [  90, 1785]])

In [22]:
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

          -1       0.51      0.94      0.66        98
           1       1.00      0.95      0.97      1875

    accuracy                           0.95      1973
   macro avg       0.75      0.95      0.82      1973
weighted avg       0.97      0.95      0.96      1973



## Support Vector Machine

In [23]:
from sklearn import svm

In [24]:
clf = svm.SVC()
clf.fit(Xtrain_vec, y_train)
clfpreds = clf.predict(Xtest_vec)

In [25]:
confusion_matrix(clfpreds,y_test)

array([[  16,    0],
       [ 166, 1791]])

In [26]:
print(classification_report(clfpreds,y_test))

              precision    recall  f1-score   support

          -1       0.09      1.00      0.16        16
           1       1.00      0.92      0.96      1957

    accuracy                           0.92      1973
   macro avg       0.54      0.96      0.56      1973
weighted avg       0.99      0.92      0.95      1973



## KNeighborsClassifier

In [27]:
from sklearn.neighbors import KNeighborsClassifier

In [28]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(Xtrain_vec, y_train)
knnpreds = knn.predict(Xtest_vec)

In [29]:
confusion_matrix(knnpreds,y_test)

array([[   0,    0],
       [ 182, 1791]])

In [30]:
print(classification_report(knnpreds,y_test))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           1       1.00      0.91      0.95      1973

    accuracy                           0.91      1973
   macro avg       0.50      0.45      0.48      1973
weighted avg       1.00      0.91      0.95      1973



/Users/lauriekaneta/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lauriekaneta/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lauriekaneta/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Multinomial Naïve Bayes 

In [31]:
from sklearn.naive_bayes import MultinomialNB

In [32]:
MNB = MultinomialNB()
MNB.fit(Xtrain_vec, y_train)
MNBpreds = MNB.predict(Xtest_vec)

In [33]:
confusion_matrix(MNBpreds,y_test)

array([[  24,    1],
       [ 158, 1790]])

In [34]:
print(classification_report(MNBpreds,y_test))

              precision    recall  f1-score   support

          -1       0.13      0.96      0.23        25
           1       1.00      0.92      0.96      1948

    accuracy                           0.92      1973
   macro avg       0.57      0.94      0.59      1973
weighted avg       0.99      0.92      0.95      1973



## Random Forest Classifier

I chose to add another model to train, the Random Forest CLassifier, since they generally perform well on imbalanced datasets. However, as we can see below from the F1 scores, it is not the best of the models trained.

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
RFC = RandomForestClassifier(random_state = 42)
RFC.fit(Xtrain_vec, y_train)
y_pred_RFC = RFC.predict(Xtest_vec)

In [37]:
confusion_matrix(y_pred_RFC,y_test)

array([[   9,    0],
       [ 173, 1791]])

In [38]:
print(classification_report(y_pred_RFC,y_test))

              precision    recall  f1-score   support

          -1       0.05      1.00      0.09         9
           1       1.00      0.91      0.95      1964

    accuracy                           0.91      1973
   macro avg       0.52      0.96      0.52      1973
weighted avg       1.00      0.91      0.95      1973



### None of these models outperformed the models that used the default, ngram_range = default=(1, 1).